In [180]:
from backtest.repository.finance.finance_repo import FinanceRepo
%load_ext autoreload
%autoreload 2

from backtest.domains.strategy import StockData
from backtest.domains.strategy import Strategy
from backtest.domains.strategy_result import StrategyResult,StrategyResultColumnType
from backtest.domains.backtest import Backtest
from backtest.repository.webrepo.crypto.bithumb_repo import BithumbRepo
from backtest.repository.webrepo.crypto.upbit_repo import UpbitRepo
from backtest.repository.webrepo.crypto.binance_repo import BinanceRepo
from backtest.use_cases.stockdata_from_repo import stockdata_from_repo
from backtest.request.stockdata_from_repo import build_stock_data_from_repo_request
from backtest.use_cases.strategy_execute import sma_function, sma_big_stock_function, sma_multi_big_stock_function, greed_fear_index_function, rsi_function, rsi_big_stock_function
from backtest.use_cases.backtest_execute import backtest_execute
from backtest.util.stockdata_util import get_greed_fear_index
import pandas as pd

request = build_stock_data_from_repo_request(
    filters={'order__eq': 'ALGO', 'from__eq': '2019-01-01'})
response = stockdata_from_repo(BithumbRepo(), request=request)

request2 = build_stock_data_from_repo_request(
    filters={'order__eq': 'BTC', 'from__eq': '2019-01-01'})
response2 = stockdata_from_repo(BithumbRepo(), request=request2)

request3 = build_stock_data_from_repo_request(
    filters={'order__eq': 'ETC', 'from__eq': '2019-01-01'})
response3 = stockdata_from_repo(BithumbRepo(), request=request3)

request4 = build_stock_data_from_repo_request(
    filters={'order__eq': 'US500', 'from__eq': '2019-11-01'})
response4 = stockdata_from_repo(FinanceRepo(), request=request4)

request5 = build_stock_data_from_repo_request(
    filters={'order__eq': 'DOT', 'from__eq': '2019-01-01'})
response5 = stockdata_from_repo(BithumbRepo(), request=request5)

stockdata1 = response.value
stockdata2 = response2.value
stockdata3 = response3.value
stockdata4 = response4.value
stockdata5 = response5.value

strategy1 = Strategy(name='snp_big_stock', function=sma_big_stock_function,
                    weight=3, options={'big_stock': stockdata4, 'rolling': 100})
strategy2 = Strategy(name='sma_big_stock', function=sma_big_stock_function,
                    weight=2, options={'big_stock': stockdata2, 'rolling': 90})
strategy3 = Strategy(name='sma_self', function=sma_function,
                    weight=1, options={'rolling': 10})
greed_fear_df = get_greed_fear_index()
strategy4 = Strategy(name='Greed_Fear_Index',function=greed_fear_index_function,
                    weight=1, options={'greed_fear_index_data': greed_fear_df,'index_fear' : 20,'index_greed':60})
strategy5 = Strategy(name='rsi_function',function=rsi_function,
                    weight=0.5, options={'period': 15,'overbought_level' : 30,'oversold_level': 70})
strategy6 = Strategy(name='rsi_big_stock_function',function=rsi_big_stock_function,
                    weight=0.5, options={'big_stock':stockdata4, 'period': 15,'overbought_level' : 30,'oversold_level':70})
strategy7 = Strategy(name='sma_multi_big_stock', function=sma_multi_big_stock_function,
                    weight=1, options={'big_stock': stockdata4, 'rolling_list': [100,15]})
strategy8 = Strategy(name='sma_multi_big_stock', function=sma_multi_big_stock_function,
                    weight=1, options={'big_stock': stockdata2, 'rolling_list': [100,15]})


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [181]:
backtest = Backtest(strategy_list=[strategy1,strategy2,strategy3,strategy4,strategy5,strategy6,strategy7,strategy8], stockdata_list=[stockdata3,stockdata5])
backtest_result = backtest_execute(backtest).value
backtest_result.value
df=backtest_result.value['total_profit'].expanding().sum()
df

date
2019-01-01    0.000000
2019-01-02    0.000000
2019-01-03    0.000000
2019-01-04    0.000000
2019-01-05    0.000000
                ...   
2023-04-05    4.701043
2023-04-06    4.701043
2023-04-07    4.701043
2023-04-08    4.701043
2023-04-09    4.701043
Freq: D, Name: total_profit, Length: 1560, dtype: float64

In [182]:
backtest_result.value['total_profit'].min()

-0.096255488293015

In [109]:
backtest_result.value.to_csv('sample.csv')

In [171]:
def calculate_rsi(data, period):
    delta = data.diff().dropna()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


df=stockdata5.data
df['RSI']=calculate_rsi(df['close'],14)
def _rsi_function(r):
    if r <= 70 and r > 70:
        return (StrategyResultColumnType.BUY, 1)
    elif r >= 30 and r < 30:
        return (StrategyResultColumnType.SELL, 1)
    else:
        return (StrategyResultColumnType.KEEP, 1)
result=df.apply(lambda r:_rsi_function(r['close']),axis=1)

In [6]:
%load_ext autoreload
%autoreload 2
from backtest.request.selector_reference_from_repo import build_selector_reference_from_repo_request
from backtest.repository.webrepo.crypto.coingecko_repo import CoinGeckoRepo
from backtest.use_cases.selector_reference_from_repo import selector_reference_from_repo

request = build_selector_reference_from_repo_request(filters={'symbol__eq':'BTC'})
result = selector_reference_from_repo(CoinGeckoRepo(),request,cache=True)
selector_reference=result.value
selector_reference.data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,marketcap
date,
2013-04-28,1.500518e+09
2013-04-29,1.575032e+09
2013-04-30,1.501657e+09
2013-05-01,1.298952e+09
2013-05-02,1.148668e+09
...,...
2023-04-16,5.869698e+11
2023-04-17,5.862682e+11
2023-04-18,5.699581e+11
